In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

# text NLP
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from wordcloud import WordCloud
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode

# Preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler 

# model
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from scipy.sparse import hstack

# Cleaning & Preprocessing

In [3]:
# import nltk
# import subprocess

# # Download and unzip wordnet
# try:
#     nltk.data.find('wordnet.zip')
# except:
#     nltk.download('wordnet', download_dir='/kaggle/working/')
#     command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
#     subprocess.run(command.split())
#     nltk.data.path.append('/kaggle/working/')

# # Now you can import the NLTK resources as usual
# from nltk.corpus import wordnet

In [4]:
# class BasicTextCleaning:
#     def __init__(self):
#         # define some necessary elements
#         self.stopwords = set(stopwords.words('english'))
#         self.words_corpus = set(words.words())
#         self.stemmer = PorterStemmer()
#         self.lemmatizer = WordNetLemmatizer()

#         # dictionary of methods can be used
#         self.methods = {'lowercase': str.lower,
#                         'accent_removal': self.accent_removal,
#                         'strip': str.strip,
#                         'nice_display': self.nice_display,
#                         'tokenization': nltk.word_tokenize,
#                         'stemming': self.stemming,
#                         'lemmatization': self.lemmatization,
#                         'punctuation_removal': self.punctuation_removal,
#                         'stopwords_removal': self.stopwords_removal,
#                         'contractions_expand': self.contractions_expand,
#                         'nonsense_removal': self.nonsense_removal,
#                         'number_removal': self.number_removal}

#         self.punctuations = '[%s]' % re.escape(string.punctuation)

#     def text_cleaning(self, texts, methods=None):
#         if not methods:
#             methods = ['accent_removal', 'lowercase', 'nice_display', 'punctuation_removal',
#                        'stopwords_removal', 'lemmatization', 'stemming']
#         if isinstance(texts, str):
#             texts = [texts]
#         cleaned_texts = []
#         for text in texts:
#             for method in methods:
#                 if method not in self.methods.keys():
#                     raise Warning('Invalid method "{}". Basic text cleaning methods available: {}'.format(method, ", ".join(self.methods.keys())))
#                 text = self.methods[method](text)
#             cleaned_texts.append(text)
#         return cleaned_texts

#     def strip_text(self, text):
#         return text.strip()

#     def lowercase(self, text):
#         return text.lower()

#     def contractions_expand(self, text):
#         return contractions.fix(text)

#     def number_removal(self, text):
#         text = re.sub(r"[^a-zA-Z]", " ", text)
#         text = re.sub(r'\s+', ' ', text)
#         return text

#     def nice_display(self, text):
#         text = re.sub(r"([^\w\s([{\'])(\w)", r"\1 \2", text)
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()

#     def accent_removal(self, text):
#         text = unidecode(text)
#         return text

#     def punctuation_removal(self, text):
#         text = re.sub(self.punctuations, ' ', text)
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()

#     def stopwords_removal(self, text):
#         return " ".join([word for word in text.split() if word not in self.stopwords])

#     def stemming(self, text):
#         return " ".join([self.stemmer.stem(word) for word in text.split()])

#     def lemmatization(self, text):
#         return " ".join([self.lemmatizer.lemmatize(word) for word in text.split()])

#     def tokenization(self, text):
#         return nltk.word_tokenize(text)

#     def nonsense_removal(self, text):
#         return " ".join([word for word in text.split() if wordnet.synsets(word)])

In [6]:
# cleaning = BasicTextCleaning()
# data['text'] = cleaning.text_cleaning(data['text_'])
# data.head()

In [2]:
data = pd.read_csv('/kaggle/input/data-cleaning/datacleaning.csv')


In [3]:
data['label'].replace('CG', 1, inplace=True)
data['label'].replace('OR', 0, inplace=True)

/tmp/ipykernel_34/384321868.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['label'].replace('CG', 1, inplace=True)
/tmp/ipykernel_34/384321868.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [4]:
data = data.fillna('')

In [6]:
data['length'] = data['text'].apply(len)

In [7]:
data.head()

,category,rating,label,text_,text,length
0,Home_and_Kitchen_5,5.0,1,"Love this! Well made, sturdy, and very comfor...",love well made sturdi comfort love pretti,41
1,Home_and_Kitchen_5,5.0,1,"love it, a great upgrade from the original. I...",love great upgrad origin mine coupl year,40
2,Home_and_Kitchen_5,5.0,1,This pillow saved my back. I love the look and...,pillow save back love look feel pillow,38
3,Home_and_Kitchen_5,1.0,1,"Missing information on how to use it, but it i...",miss inform use great product price,35
4,Home_and_Kitchen_5,5.0,1,Very nice set. Good quality. We have had the s...,nice set good qualiti set two month,35


# Save result

In [6]:
# data_save = pd.DataFrame(columns=['data','length_used', 'feature_extraction', 'feature_selection', 'model', 'accuracy', 'f1', 'recall', 'precision', 'roc_auc', 'notes'])
# data_save.to_csv('results.csv', index=False)

In [36]:
def save_and_print(data, length_used, feature_extraction, feature_selection, model, scores):
    data_save = pd.read_csv('/kaggle/working/results.csv')
    for feature_extraction in feature_extractions:
        sc = str(feature_extraction)
        new_row = {'data': data, 'length_used': length_used, 
               'feature_extraction': feature_extraction, 
               'feature_selection': feature_selection, 
               'model': model, 'accuracy': np.mean(scores[sc]['accuracy']),
               'f1': np.mean(scores[sc]['f1']), 'recall': np.mean(scores[sc]['recall']), 
               'precision': np.mean(scores[sc]['precision']), 'roc_auc': np.mean(scores[sc]['roc_auc']), 
               'notes': notes}
        data_save.loc[len(data_save)] = new_row
        data_save.to_csv('/kaggle/working/results.csv', index=False)

In [10]:
X = data[['text', 'length']]
y = data['label']

# Case 1: Use length

In [ ]:
X = data[['text', 'length']]
y = data['label']

In [12]:
dataname = 'Fake Review Dataset'
lengh_used = 'minmaxscale'
feature_selection = None
notes = None

# logistic

In [37]:
feature_extractions = [
    HashingVectorizer(ngram_range=(1, 1)),
    HashingVectorizer(ngram_range=(1, 2)),
    HashingVectorizer(ngram_range=(1, 3))
]

In [17]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [
    HashingVectorizer(ngram_range=(1, 1)),
    HashingVectorizer(ngram_range=(1, 2)),
    HashingVectorizer(ngram_range=(1, 3))
]

length_scaler = MinMaxScaler()
scores_lr = {}

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_text = X_train['text']
    X_test_text = X_test['text']
    X_train_length = X_train[['length']]
    X_test_length = X_test[['length']]
    
    X_train_length_scaled = length_scaler.fit_transform(X_train_length)
    X_test_length_scaled = length_scaler.transform(X_test_length)
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train_text)
        X_test_hashed = vectorizer.transform(X_test_text)
        
        # Combine hashed features with length features
        X_train_combined = hstack((X_train_hashed, X_train_length_scaled))
        X_test_combined = hstack((X_test_hashed, X_test_length_scaled))
        
        model_lr = LogisticRegression(max_iter=1000)
        model_lr.fit(X_train_combined, y_train)
        
        y_test_pred = model_lr.predict(X_test_combined)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lr.predict_proba(X_test_combined)[:, 1])
        
        key = str(feature_extraction)
        if key not in scores_lr:
            scores_lr[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}
        
        scores_lr[key]['accuracy'].append(accuracy)
        scores_lr[key]['f1'].append(f1)
        scores_lr[key]['recall'].append(recall)
        scores_lr[key]['precision'].append(precision)
        scores_lr[key]['roc_auc'].append(roc_auc)

In [38]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lr, scores_lr)

# KNN

In [39]:
feature_extractions = [HashingVectorizer(ngram_range=(1, 2))]

In [21]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 2))]

length_scaler = MinMaxScaler()
scores_knn = {}

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_text = X_train['text']
    X_test_text = X_test['text']
    X_train_length = X_train[['length']]
    X_test_length = X_test[['length']]
    
    X_train_length_scaled = length_scaler.fit_transform(X_train_length)
    X_test_length_scaled = length_scaler.transform(X_test_length)
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train_text)
        X_test_hashed = vectorizer.transform(X_test_text)
        
        # Combine hashed features with length features
        X_train_combined = hstack((X_train_hashed, X_train_length_scaled))
        X_test_combined = hstack((X_test_hashed, X_test_length_scaled))
        
        model_knn = KNeighborsClassifier()
        model_knn.fit(X_train_combined, y_train)
        
        y_test_pred = model_knn.predict(X_test_combined)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_combined)[:, 1])
        
        key = str(feature_extraction)
        if key not in scores_knn:
            scores_knn[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}
        
        scores_knn[key]['accuracy'].append(accuracy)
        scores_knn[key]['f1'].append(f1)
        scores_knn[key]['recall'].append(recall)
        scores_knn[key]['precision'].append(precision)
        scores_knn[key]['roc_auc'].append(roc_auc)

In [40]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_knn, scores_knn)

In [25]:
import catboost as cb
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
import lightgbm as lgb

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


# LightGBM

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 2))]

length_scaler = MinMaxScaler()
scores_lgbm = {}

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_text = X_train['text']
    X_test_text = X_test['text']
    X_train_length = X_train[['length']]
    X_test_length = X_test[['length']]
    
    X_train_length_scaled = length_scaler.fit_transform(X_train_length)
    X_test_length_scaled = length_scaler.transform(X_test_length)
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train_text)
        X_test_hashed = vectorizer.transform(X_test_text)
        
        # Combine hashed features with length features
        X_train_combined = hstack((X_train_hashed, X_train_length_scaled))
        X_test_combined = hstack((X_test_hashed, X_test_length_scaled))
        
        model_lgbm = lgb.LGBMClassifier()
        model_lgbm.fit(X_train_combined, y_train)
        
        y_test_pred = model_lgbm.predict(X_test_combined)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lgbm.predict_proba(X_test_combined)[:, 1])
        
        key = str(feature_extraction)
        if key not in scores_lgbm:
            scores_lgbm[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}
        
        scores_lgbm[key]['accuracy'].append(accuracy)
        scores_lgbm[key]['f1'].append(f1)
        scores_lgbm[key]['recall'].append(recall)
        scores_lgbm[key]['precision'].append(precision)
        scores_lgbm[key]['roc_auc'].append(roc_auc)

[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.320004 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 234120
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 7458
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


[LightGBM] [Info] Number of positive: 16081, number of negative: 16264
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.352111 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234960
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 7467
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497171 -> initscore=-0.011316
[LightGBM] [Info] Start training from score -0.011316


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


[LightGBM] [Info] Number of positive: 16232, number of negative: 16114
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.343592 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 232540
[LightGBM] [Info] Number of data points in the train set: 32346, number of used features: 7413
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501824 -> initscore=0.007296
[LightGBM] [Info] Start training from score 0.007296


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


[LightGBM] [Info] Number of positive: 16159, number of negative: 16187
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.397081 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 234960
[LightGBM] [Info] Number of data points in the train set: 32346, number of used features: 7486
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499567 -> initscore=-0.001731
[LightGBM] [Info] Start training from score -0.001731


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


[LightGBM] [Info] Number of positive: 16192, number of negative: 16154
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.489974 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 233714
[LightGBM] [Info] Number of data points in the train set: 32346, number of used features: 7439
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500587 -> initscore=0.002350
[LightGBM] [Info] Start training from score 0.002350


/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1192: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [41]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lgbm, scores_lgbm)

# XGBoost

In [29]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 2))]

length_scaler = MinMaxScaler()
scores_xg = {}

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_text = X_train['text']
    X_test_text = X_test['text']
    X_train_length = X_train[['length']]
    X_test_length = X_test[['length']]
    
    X_train_length_scaled = length_scaler.fit_transform(X_train_length)
    X_test_length_scaled = length_scaler.transform(X_test_length)
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train_text)
        X_test_hashed = vectorizer.transform(X_test_text)
        
        # Combine hashed features with length features
        X_train_combined = hstack((X_train_hashed, X_train_length_scaled))
        X_test_combined = hstack((X_test_hashed, X_test_length_scaled))
        
        model_xg = XGBClassifier()
        model_xg.fit(X_train_combined, y_train)
        
        y_test_pred = model_xg.predict(X_test_combined)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_xg.predict_proba(X_test_combined)[:, 1])
        
        key = str(feature_extraction)
        if key not in scores_xg:
            scores_xg[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}
        
        scores_xg[key]['accuracy'].append(accuracy)
        scores_xg[key]['f1'].append(f1)
        scores_xg[key]['recall'].append(recall)
        scores_xg[key]['precision'].append(precision)
        scores_xg[key]['roc_auc'].append(roc_auc)

In [42]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_xg, scores_xg)

# ADABoost

In [44]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 2))]

length_scaler = MinMaxScaler()
scores_ada = {}

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train_text = X_train['text']
    X_test_text = X_test['text']
    X_train_length = X_train[['length']]
    X_test_length = X_test[['length']]
    
    X_train_length_scaled = length_scaler.fit_transform(X_train_length)
    X_test_length_scaled = length_scaler.transform(X_test_length)
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train_text)
        X_test_hashed = vectorizer.transform(X_test_text)
        
        # Combine hashed features with length features
        X_train_combined = hstack((X_train_hashed, X_train_length_scaled))
        X_test_combined = hstack((X_test_hashed, X_test_length_scaled))
        
        model_ada = AdaBoostClassifier()
        model_ada.fit(X_train_combined, y_train)
        
        y_test_pred = model_ada.predict(X_test_combined)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_ada.predict_proba(X_test_combined)[:, 1])
        
        key = str(feature_extraction)
        if key not in scores_ada:
            scores_ada[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}
        
        scores_ada[key]['accuracy'].append(accuracy)
        scores_ada[key]['f1'].append(f1)
        scores_ada[key]['recall'].append(recall)
        scores_ada[key]['precision'].append(precision)
        scores_ada[key]['roc_auc'].append(roc_auc)

In [45]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_ada, scores_ada)

# CatBoost

# Case 2
Don't use feature lengh, feature selection

In [ ]:
X = data['text']
y = data['label']

In [7]:
dataname = 'Fake Review Dataset'
lengh_used = None
feature_selection = None
notes = None

# Ngram = (1,1)

# LGBM

In [8]:
import lightgbm as lgb

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [40]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lg = lgb.LGBMClassifier(force_row_wise=True)
        model_lg.fit(X_train_hashed, y_train)

        y_test_pred = model_lg.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lg.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores:
            scores[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores[key]['accuracy'].append(accuracy)
        scores[key]['f1'].append(f1)
        scores[key]['recall'].append(recall)
        scores[key]['precision'].append(precision)
        scores[key]['roc_auc'].append(roc_auc)

[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 25499
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightG

In [53]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lg, scores)

# XG boost

In [9]:
from xgboost import XGBClassifier

In [44]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_xgb = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_xgb = XGBClassifier()
        model_xgb.fit(X_train_hashed, y_train)

        y_test_pred = model_xgb.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_xgb.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_xgb:
            scores_xgb[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_xgb[key]['accuracy'].append(accuracy)
        scores_xgb[key]['f1'].append(f1)
        scores_xgb[key]['recall'].append(recall)
        scores_xgb[key]['precision'].append(precision)
        scores_xgb[key]['roc_auc'].append(roc_auc)

In [56]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_xgb, scores_xgb)

# Adaboost

In [9]:
from sklearn.ensemble import AdaBoostClassifier

In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_ada = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_ada = AdaBoostClassifier(n_estimators=100, learning_rate=1.0)
        model_ada.fit(X_train_hashed, y_train)

        y_test_pred = model_ada.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_ada.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_ada:
            scores_ada[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_ada[key]['accuracy'].append(accuracy)
        scores_ada[key]['f1'].append(f1)
        scores_ada[key]['recall'].append(recall)
        scores_ada[key]['precision'].append(precision)
        scores_ada[key]['roc_auc'].append(roc_auc)

In [29]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_ada, scores_ada)

# CATBOOST

In [8]:
import catboost as cb

In [24]:
folds = kf.split(X, y)
scores_cat = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_cat = cb.CatBoostClassifier(
            random_state=42,
            task_type='GPU',
            border_count=2 * len(feature_extractions)
        )

        model_cat.fit(
            X_train_hashed,
            y_train,
            verbose=False,
            plot=False,
            early_stopping_rounds=50,
            use_best_model=True,
            eval_set=(X_test_hashed, y_test)
        )

        y_test_pred = model_cat.predict(X_test_hashed)

        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_cat.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_cat:
            scores_cat[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_cat[key]['accuracy'].append(accuracy)
        scores_cat[key]['f1'].append(f1)
        scores_cat[key]['recall'].append(recall)
        scores_cat[key]['precision'].append(precision)
        scores_cat[key]['roc_auc'].append(roc_auc)

In [27]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_cat, scores_cat)

# Logistic Regression

In [11]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]

folds = kf.split(X, y)
scores_lr = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lr = LogisticRegression(max_iter = 1000)
        model_lr.fit(X_train_hashed, y_train)

        y_test_pred = model_lr.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lr.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_lr:
            scores_lr[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_lr[key]['accuracy'].append(accuracy)
        scores_lr[key]['f1'].append(f1)
        scores_lr[key]['recall'].append(recall)
        scores_lr[key]['precision'].append(precision)
        scores_lr[key]['roc_auc'].append(roc_auc)

In [12]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lr, scores_lr)

# KNN

In [9]:
# KNN default with ngram = 1,1
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer()]
metrics_param = ['cosine','minkowski']
neighbors_param = [5, 10, 50, 100]
folds = kf.split(X, y)
scores_knn = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)
#         for metric in metrics_param:
#             for neighbor in neighbors_param:
        model_knn = KNeighborsClassifier()
        model_knn.fit(X_train_hashed, y_train)

        y_test_pred = model_knn.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_knn:
            scores_knn[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_knn[key]['accuracy'].append(accuracy)
        scores_knn[key]['f1'].append(f1)
        scores_knn[key]['recall'].append(recall)
        scores_knn[key]['precision'].append(precision)
        scores_knn[key]['roc_auc'].append(roc_auc)

In [12]:
a = np.mean(scores_knn['HashingVectorizer()']['accuracy'])
b = np.mean(scores_knn['HashingVectorizer()']['f1'])
c = np.mean(scores_knn['HashingVectorizer()']['roc_auc'])
print(a,b,c)

0.5694739507509556 0.6939923442843808 0.7362644164722172


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10), HashingVectorizer(n_features = 50),
                     HashingVectorizer(n_features = 100),HashingVectorizer()]
metrics_param = ['cosine','minkowski']
neighbors_param = [5, 10, 50, 100]
folds = kf.split(X, y)
scores_knn = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)
        for metric in metrics_param:
            for neighbor in neighbors_param:
                model_knn = KNeighborsClassifier(metric = metric, n_neighbors = neighbor)
                model_knn.fit(X_train_hashed, y_train)

                y_test_pred = model_knn.predict(X_test_hashed)
                accuracy = accuracy_score(y_test, y_test_pred)
                f1 = f1_score(y_test, y_test_pred, pos_label=1)
                recall = recall_score(y_test, y_test_pred)
                precision = precision_score(y_test, y_test_pred)
                roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_hashed)[:, 1])

                key = str(feature_extraction)
                if key not in scores_knn:
                    scores_knn[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

                scores_knn[key]['accuracy'].append(accuracy)
                scores_knn[key]['f1'].append(f1)
                scores_knn[key]['recall'].append(recall)
                scores_knn[key]['precision'].append(precision)
                scores_knn[key]['roc_auc'].append(roc_auc)

In [ ]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_knn, scores_knn)

# Ngram = (1,2)

# KNN

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 2)), HashingVectorizer(ngram_range=(1, 3))]

folds = kf.split(X, y)
scores_knn = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_knn = KNeighborsClassifier()
        model_knn.fit(X_train_hashed, y_train)

        y_test_pred = model_knn.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_knn:
            scores_knn[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_knn[key]['accuracy'].append(accuracy)
        scores_knn[key]['f1'].append(f1)
        scores_knn[key]['recall'].append(recall)
        scores_knn[key]['precision'].append(precision)
        scores_knn[key]['roc_auc'].append(roc_auc)

In [ ]:
a1 = np.mean(scores_knn['HashingVectorizer()']['accuracy'])
b1 = np.mean(scores_knn['HashingVectorizer()']['f1'])
c1 = np.mean(scores_knn['HashingVectorizer()']['roc_auc'])
print(a1,b1,c1)

# Logistic 

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 2)), 
                       HashingVectorizer(n_features = 50, ngram_range=(1, 2)),
                     HashingVectorizer(n_features = 100, ngram_range=(1, 2)),
                       HashingVectorizer(ngram_range=(1, 2))]

folds = kf.split(X, y)
scores_lr = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lr = LogisticRegression(max_iter = 1000)
        model_lr.fit(X_train_hashed, y_train)

        y_test_pred = model_lr.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lr.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_lr:
            scores_lr[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_lr[key]['accuracy'].append(accuracy)
        scores_lr[key]['f1'].append(f1)
        scores_lr[key]['recall'].append(recall)
        scores_lr[key]['precision'].append(precision)
        scores_lr[key]['roc_auc'].append(roc_auc)

In [10]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lr, scores_lr)

# LGBM

In [18]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 2)), 
                       HashingVectorizer(n_features = 50, ngram_range=(1, 2)),
                     HashingVectorizer(n_features = 100, ngram_range=(1, 2)),
                       HashingVectorizer(ngram_range=(1, 2))]

folds = kf.split(X, y)
scores = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lg = lgb.LGBMClassifier(force_row_wise=True)
        model_lg.fit(X_train_hashed, y_train)

        y_test_pred = model_lg.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lg.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores:
            scores[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores[key]['accuracy'].append(accuracy)
        scores[key]['f1'].append(f1)
        scores[key]['recall'].append(recall)
        scores[key]['precision'].append(precision)
        scores[key]['roc_auc'].append(roc_auc)

[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightG

In [19]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lg, scores)

# XGboost

In [21]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 2)), 
                       HashingVectorizer(n_features = 50, ngram_range=(1, 2)),
                     HashingVectorizer(n_features = 100, ngram_range=(1, 2)),
                       HashingVectorizer(ngram_range=(1, 2))]

folds = kf.split(X, y)
scores_xgb = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_xgb = XGBClassifier()
        model_xgb.fit(X_train_hashed, y_train)

        y_test_pred = model_xgb.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_xgb.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_xgb:
            scores_xgb[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_xgb[key]['accuracy'].append(accuracy)
        scores_xgb[key]['f1'].append(f1)
        scores_xgb[key]['recall'].append(recall)
        scores_xgb[key]['precision'].append(precision)
        scores_xgb[key]['roc_auc'].append(roc_auc)

In [22]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_xgb, scores_xgb)

# ADA Boost

In [24]:
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 2)), 
#                        HashingVectorizer(n_features = 50, ngram_range=(1, 2)),
#                      HashingVectorizer(n_features = 100, ngram_range=(1, 2)),
#                        HashingVectorizer(ngram_range=(1, 2))]

folds = kf.split(X, y)
scores_ada = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_ada = AdaBoostClassifier(n_estimators=100, learning_rate=1.0)
        model_ada.fit(X_train_hashed, y_train)

        y_test_pred = model_ada.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_ada.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_ada:
            scores_ada[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_ada[key]['accuracy'].append(accuracy)
        scores_ada[key]['f1'].append(f1)
        scores_ada[key]['recall'].append(recall)
        scores_ada[key]['precision'].append(precision)
        scores_ada[key]['roc_auc'].append(roc_auc)

In [26]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_ada, scores_ada)

# CAT boost

In [12]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 2)), 
                       HashingVectorizer(n_features = 50, ngram_range=(1, 2)),
                     HashingVectorizer(n_features = 100, ngram_range=(1, 2)),
                       HashingVectorizer(ngram_range=(1, 2))]

folds = kf.split(X, y)
scores_cat = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_cat = cb.CatBoostClassifier(
            random_state=42,
            task_type='GPU',
            border_count=2 * len(feature_extractions)
        )

        model_cat.fit(
            X_train_hashed,
            y_train,
            verbose=False,
            plot=False,
            early_stopping_rounds=50,
            use_best_model=True,
            eval_set=(X_test_hashed, y_test)
        )

        y_test_pred = model_cat.predict(X_test_hashed)

        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_cat.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_cat:
            scores_cat[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_cat[key]['accuracy'].append(accuracy)
        scores_cat[key]['f1'].append(f1)
        scores_cat[key]['recall'].append(recall)
        scores_cat[key]['precision'].append(precision)
        scores_cat[key]['roc_auc'].append(roc_auc)

In [13]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_cat, scores_cat)

# Ngram = (1,3)

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(n_features = 10, ngram_range=(1, 3)), 
                       HashingVectorizer(n_features = 50, ngram_range=(1, 3)),
                     HashingVectorizer(n_features = 100, ngram_range=(1, 3)),
                       HashingVectorizer(ngram_range=(1, 3))]

# KNN

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

feature_extractions = [HashingVectorizer(ngram_range=(1, 3))]

folds = kf.split(X, y)
scores_knn = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_knn = KNeighborsClassifier()
        model_knn.fit(X_train_hashed, y_train)

        y_test_pred = model_knn.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_knn.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores:
            scores[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_knn[key]['accuracy'].append(accuracy)
        scores_knn[key]['f1'].append(f1)
        scores_knn[key]['recall'].append(recall)
        scores_knn[key]['precision'].append(precision)
        scores_knn[key]['roc_auc'].append(roc_auc)

# Logistic

In [16]:
folds = kf.split(X, y)
scores_lr = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lr = LogisticRegression(max_iter = 1000)
        model_lr.fit(X_train_hashed, y_train)

        y_test_pred = model_lr.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lr.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_lr:
            scores_lr[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_lr[key]['accuracy'].append(accuracy)
        scores_lr[key]['f1'].append(f1)
        scores_lr[key]['recall'].append(recall)
        scores_lr[key]['precision'].append(precision)
        scores_lr[key]['roc_auc'].append(roc_auc)

In [18]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lr, scores_lr)

# LGBM

In [20]:
folds = kf.split(X, y)
scores = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_lg = lgb.LGBMClassifier(force_row_wise=True)
        model_lg.fit(X_train_hashed, y_train)

        y_test_pred = model_lg.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_lg.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores:
            scores[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores[key]['accuracy'].append(accuracy)
        scores[key]['f1'].append(f1)
        scores[key]['recall'].append(recall)
        scores[key]['precision'].append(precision)
        scores[key]['roc_auc'].append(roc_auc)

[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 12750
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightGBM] [Info] Number of positive: 16200, number of negative: 16145
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 32345, number of used features: 100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500850 -> initscore=0.003401
[LightGBM] [Info] Start training from score 0.003401
[LightG

In [21]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_lg, scores)

# XG Boost

In [23]:
folds = kf.split(X, y)
scores_xgb = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_xgb = XGBClassifier()
        model_xgb.fit(X_train_hashed, y_train)

        y_test_pred = model_xgb.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_xgb.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_xgb:
            scores_xgb[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_xgb[key]['accuracy'].append(accuracy)
        scores_xgb[key]['f1'].append(f1)
        scores_xgb[key]['recall'].append(recall)
        scores_xgb[key]['precision'].append(precision)
        scores_xgb[key]['roc_auc'].append(roc_auc)

In [24]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_xgb, scores_xgb)

# ADA Boost

In [ ]:
folds = kf.split(X, y)
scores_ada = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_ada = AdaBoostClassifier(n_estimators=100, learning_rate=1.0)
        model_ada.fit(X_train_hashed, y_train)

        y_test_pred = model_ada.predict(X_test_hashed)
        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_ada.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_ada:
            scores_ada[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_ada[key]['accuracy'].append(accuracy)
        scores_ada[key]['f1'].append(f1)
        scores_ada[key]['recall'].append(recall)
        scores_ada[key]['precision'].append(precision)
        scores_ada[key]['roc_auc'].append(roc_auc)

In [ ]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_ada, scores_ada)

# CAT Boost

In [ ]:
folds = kf.split(X, y)
scores_cat = {}
for train_index, test_index in folds:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for feature_extraction in feature_extractions:
        vectorizer = feature_extraction
        X_train_hashed = vectorizer.fit_transform(X_train)
        X_test_hashed = vectorizer.transform(X_test)

        model_cat = cb.CatBoostClassifier(
            random_state=42,
            task_type='GPU',
            border_count=2 * len(feature_extractions)
        )

        model_cat.fit(
            X_train_hashed,
            y_train,
            verbose=False,
            plot=False,
            early_stopping_rounds=50,
            use_best_model=True,
            eval_set=(X_test_hashed, y_test)
        )

        y_test_pred = model_cat.predict(X_test_hashed)

        accuracy = accuracy_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred, pos_label=1)
        recall = recall_score(y_test, y_test_pred)
        precision = precision_score(y_test, y_test_pred)
        roc_auc = roc_auc_score(y_test, model_cat.predict_proba(X_test_hashed)[:, 1])

        key = str(feature_extraction)
        if key not in scores_cat:
            scores_cat[key] = {'accuracy': [], 'f1': [], 'recall': [], 'precision': [], 'roc_auc': []}

        scores_cat[key]['accuracy'].append(accuracy)
        scores_cat[key]['f1'].append(f1)
        scores_cat[key]['recall'].append(recall)
        scores_cat[key]['precision'].append(precision)
        scores_cat[key]['roc_auc'].append(roc_auc)

In [ ]:
save_and_print(dataname, lengh_used, feature_extractions, feature_selection, model_cat, scores_cat)

# Result

In [46]:
result = pd.read_csv('/kaggle/working/results.csv')
# result = result.sort_values(by='accuracy', ascending=False)
result.tail(10)

,data,length_used,feature_extraction,feature_selection,model,accuracy,f1,recall,precision,roc_auc,notes
77,Fake Review Dataset,NaN,"HashingVectorizer(n_features=100, ngram_range=...",NaN,AdaBoostClassifier(n_estimators=100),0.684865,0.686397,0.684461,0.688559,0.748474,NaN
78,Fake Review Dataset,NaN,"HashingVectorizer(ngram_range=(1, 3))",NaN,AdaBoostClassifier(n_estimators=100),0.797329,0.792084,0.777390,0.807344,0.889228,NaN
79,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,LogisticRegression(max_iter=1000),0.875767,0.874141,0.863011,0.885578,0.947730,NaN
80,Fake Review Dataset,minmaxscale,HashingVectorizer(),NaN,LogisticRegression(max_iter=1000),0.853408,0.853843,0.856464,0.851254,0.931868,NaN
81,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,LogisticRegression(max_iter=1000),0.875767,0.874141,0.863011,0.885578,0.947730,NaN
82,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 3))",NaN,LogisticRegression(max_iter=1000),0.873516,0.871270,0.856277,0.886818,0.945084,NaN
83,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,KNeighborsClassifier(),0.570809,0.696239,0.983356,0.539114,0.747059,NaN
84,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,LGBMClassifier(),0.862510,0.860396,0.847538,0.873678,0.942933,NaN
85,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,"XGBClassifier(base_score=None, booster=None, c...",0.854719,0.851104,0.830525,0.872743,0.937047,NaN
86,Fake Review Dataset,minmaxscale,"HashingVectorizer(ngram_range=(1, 2))",NaN,AdaBoostClassifier(),0.755169,0.753621,0.749054,0.758277,0.847452,NaN
